In [1]:
# 大模型下载
# https://github.com/THUDM/ChatGLM-6B
# 微调lora权重在model文件夹中

import pandas as pd

train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/testB.csv')



In [2]:
res = []

for i in range(len(train_df)):
    paper_item = train_df.loc[i]
    tmp = {
    "instruction": "Please judge whether it is a medical field paper according to the given paper title and abstract, output 1 or 0, the following is the paper title, author and abstract -->",
    "input": f"title:{paper_item[1]},abstract:{paper_item[3]}",
    "output": str(paper_item[5])
  }
    res.append(tmp)

import json

with open('paper_label.json', mode='w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)



In [3]:
# 加载训练好的LoRA权重，进行预测

from peft import PeftModel
from transformers import AutoTokenizer, AutoModel, GenerationConfig, AutoModelForCausalLM

# model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model_path = "THUDM/chatglm2-6b"
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).float()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# 加载lora权重
model = PeftModel.from_pretrained(model, 'huanhuan-chat-master/output/label_xfg').float()
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
response

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

'你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。'

In [4]:
# 预测函数

def predict(text):
    response, history = model.chat(tokenizer, f"Please judge whether it is a medical field paper according to the given paper title and abstract, output 1 or 0, the following is the paper title, author and abstract -->{text}", history=[],
    temperature=0.01)
    return response




In [5]:
# 预测测试集
from tqdm import tqdm

label = []

for i in tqdm(range(len(test_df))):
    test_item = test_df.loc[i]
    test_input = f"title:{test_item[1]},author:{test_item[2]},abstract:{test_item[3]}"
    tmp = 3
    try :
        tmp = int(predict(test_input))
    except:
        pass
    label.append(tmp)

test_df['label'] = label
submit = test_df[['uuid', 'label']]
submit.to_csv('submit.csv', index=False)




  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 1/2000 [00:29<16:31:16, 29.75s/it]

1


  0%|          | 2/2000 [00:46<12:13:57, 22.04s/it]

1


In [ ]:

def predict2(text):
    response, history = model.chat(tokenizer, f"Please give 35 Keywords according to the given paper title and abstract, output the keywords and split by \";\", the following is the paper title, author and abstract -->{text}", history=[],
    temperature=0.01)
    return response


# 预测测试集
from tqdm import tqdm


keywords = []

for i in tqdm(range(len(test_df))):
    test_item = test_df.loc[i]
    test_input = f"title:{test_item[1]},author:{test_item[2]},abstract:{test_item[3]}"
    word = "null"
    try :
        word = predict2(test_input)
        print(word)
    except:
        pass
    keywords.append(word)


test_df['Keywords'] = keywords
submit = test_df[['uuid', 'Keywords', 'label']]
submit.to_csv('submit2.csv', index=False)



